In [1]:
import os
os.chdir('../')
%pwd

'c:\\PEC-26\\MLOPS - Journey\\Projects\\End-to-End-Wine-Quality-Prediction'

In [2]:
#Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    random_state: int
    target_column: str

In [3]:
#configuration
from wine_quality.constants import *
from wine_quality.utils.common import read_yaml, create_directories
from pathlib import Path

class ConfiguartionManager:
    def __init__(self, config_filepath: Path = CONFIG_FILE_PATH, params_filepath: Path = PARAMS_FILE_PATH, schema_filepath: Path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def getModelTrainerConfig(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        return ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path, 
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            random_state=params.random_state,
            target_column = schema.name
        )

In [4]:
#component
import pandas as pd
import os
from wine_quality import logger
from sklearn.linear_model import ElasticNet
import joblib

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        # Load data
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop([self.config.target_column], axis = 1)
        X_test = test_data.drop([self.config.target_column], axis = 1)
        y_train = train_data[self.config.target_column]
        y_test = test_data[self.config.target_column]

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=self.config.random_state)
        lr.fit(X_train, y_train)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [5]:
#pipeline
try:
    config = ConfiguartionManager()
    model_trainer_config = config.getModelTrainerConfig()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-01-09 11:40:53,119: INFO: common yaml file: config\config.yaml loaded successfully]
[2025-01-09 11:40:53,121: INFO: common yaml file: params.yaml loaded successfully]
[2025-01-09 11:40:53,124: INFO: common yaml file: schema.yaml loaded successfully]
[2025-01-09 11:40:53,126: INFO: common created directory at: artifacts]
[2025-01-09 11:40:53,127: INFO: common created directory at: artifacts/model_trainer]
